In [17]:
with open('../dicts/all_omographs_test.hmg', 'r') as f:
    lines = list(filter(lambda el: not el[0:2] == '||', f.readlines()))

lines = list(map(lambda el: '||' + el, lines))

with open('../dicts/all_omographs_test_out.hmg', 'w') as out:
    out.write(''.join(lines))

In [1]:
import multiprocessing
from joblib import Parallel, delayed
import re
import itertools
import time
import random

num_cores = multiprocessing.cpu_count()

In [7]:
with open("../dicts/all_omographs.hmg", 'r') as f:
    dictEntr = list(f.readlines())
with open('../text/fanfics.txt', 'r') as f:
    text = f.read().replace('\xa0', ' ')
text_unsigned = text.replace('!', '.').replace('?', '.').replace('…', '.').replace('–', '-').replace('—', '-').replace('. -', ', -').replace('ё', 'е').replace('Ё', 'Е')


In [14]:
text = "они попадают в организм…\nпродолжить жизнь без тела носителя…"
text_unsigned = text.replace('!', '.').replace('?', '.').replace('…', '.').replace('–', '-').replace('—', '-').replace('. -', ', -').replace('ё', 'е').replace('Ё', 'Е')


In [15]:

def getRegions(start, end):
    before = text_unsigned[max((start - 301), text_unsigned.rfind('\n', 0, start - 1) + 1):(start - 1)]
    after = text_unsigned[end:min((end + 301), text_unsigned.find('\n', end + 1))]
    dotFirst = max(before.rfind('.'), 0)
    dotLast = after.find('.')
    if dotLast == -1:
        dotLast = len(after)

    while len(before) - dotFirst + dotLast < 40:
        dotFirstNext = before.rfind('.', 0, dotFirst)
        dotLastNext = after.find('.', dotLast + 1)
        if dotLastNext == -1 and dotFirstNext == -1:
            break
        if dotLastNext == -1:
            dotFirst = dotFirstNext
        elif dotFirstNext == -1:
            dotLast = dotLastNext
        elif dotLastNext - dotLast > dotFirst - dotFirstNext:
            dotFirst = dotFirstNext
        else:
            dotLast = dotLastNext
            
    return [before[dotFirst:], after[1:dotLast]]
    
    
def findOccur(ind, line):
    pos = []
    if line[0:2] == '||':
        word = line[2:line.find("=")]
        if text.find(word) != -1:
            occur = re.finditer('\\b' + word + '\\b', text)
            pos = list(map(lambda o: [ind, o.start(), getRegions(o.start(), o.end()) ], occur))
    return pos

if __name__ == "__main__":
    allOcs = Parallel(n_jobs = num_cores)(delayed(findOccur)(ind, line) for ind, line in enumerate(dictEntr[:-1]))

allOcs = list(itertools.chain.from_iterable(allOcs))

In [16]:
allOcs

[[15147, 4, ['они', 'в организм']],
 [21573, 46, ['продолжить жизнь без', 'носителя']]]

In [11]:
opts = dictEntr[pr[0]][(dictEntr[pr[0]].find('=') + 1):-1].split(',')
scores = [0] * len(opts)
insts = [0] * len(opts)
opt = 1
st = ''

before = dictEntr[pr[0] + 2 + opt*3][(dictEntr[pr[0] + 2 + opt*3].find(':') + 1):-1]
after = dictEntr[pr[0] + 3 + opt*3][(dictEntr[pr[0] + 3 + opt*3].find(':') + 1):-1]
insts[opt] = before.count('|') + after.count('|')
before_log = pr[2][0]
after_log = pr[2][1]

m = re.search(before, " " + pr[2][0], re.I)
print(m)

NameError: name 'pr' is not defined

In [17]:
manager = multiprocessing.Manager()
log_dict = manager.dict({})

def makeDecision(pr, n):
    opts = dictEntr[pr[0]][(dictEntr[pr[0]].find('=') + 1):-1].split(',')
    rand = random.randint(1, n)
    if dictEntr[pr[0] + 1][0:2] == '||':
        if rand == 1:
            log_dict[len(log_dict)] = pr[2][0] + ' _' + opts[0] + '_ ' + pr[2][1] + '\n'
        return opts[0]
    scores = [0] * len(opts)
    insts = [0] * len(opts)
    opt = 0
    st = ''
    while opt < len(opts):
        before = dictEntr[pr[0] + 2 + opt*3][(dictEntr[pr[0] + 2 + opt*3].find(':') + 1):-1]
        after = dictEntr[pr[0] + 3 + opt*3][(dictEntr[pr[0] + 3 + opt*3].find(':') + 1):-1]
        insts[opt] = before.count('|') + after.count('|')
        before_log = pr[2][0]
        after_log = pr[2][1]
        if len(before) > 0:
            for o in re.finditer(before, pr[2][0], re.I):
                scores[opt] += (o.end() - o.start()) * max((20 - len(pr[2][0]) + o.end()), 1)^2
                if rand == 1:
                    before_log = before_log[:o.start()] + before_log[o.start():o.end()].upper() + before_log[o.end():]
        if len(after) > 0 :
            for o in re.finditer(after, pr[2][1], re.I):
                scores[opt] += (o.end() - o.start()) * max(20 - o.start() + 1, 1)^2
                if rand == 1:
                    after_log = after_log[:o.start()] + after_log[o.start():o.end()].upper() + after_log[o.end():]
        if rand == 1:
            st += str(scores[opt]) + ': ' + before_log + ' _' + opts[opt] + '_ ' + after_log + '\n'

        opt += 1
    if rand == 1:
        log_dict[len(log_dict)] = st
    maxScore = max(scores)
    maxInd = [i for i, j in enumerate(scores) if j == maxScore]
    resInd = maxInd[0]
    for ind in maxInd[1:]:
        if insts[ind] > insts[resInd]:
            resInd = ind
    return opts[resInd]

if __name__ == "__main__":
    descs = Parallel(n_jobs = num_cores)(delayed(makeDecision)(pr, 25) for pr in allOcs)


In [20]:
descs
allOcs

[[15147, 4, ['они', 'в организм']],
 [21573, 46, ['продолжить жизнь без', 'носителя']]]

In [40]:
allOcs.sort(key = lambda el: el[1])
newText = ''
pos = 0

for pr in allOcs:
    newText += text[pos:pr[1]] + descs[1]
    pos = len(newText)
newText += text[pos:]

In [26]:
pr = allOcs[0]
newText += text[0:4] + descs[0]
newText
pos = len(newText) + 1

In [39]:
text

'они попадают в организм…\nпродолжить жизнь без тела носителя…'

In [41]:
newText

'они тЕладают в организм…\nпродолжить жизнь без тЕла носителя…'

In [60]:
with open('log.txt', 'w') as out:
    for key in log_dict:
        out.write(log_dict[key])
        out.write('\n-----------\n\n')

In [2]:
text = 'становился все громче и громче…'
pattern = r'\b((\w+)(:? и \2)|\w+ла|\w+ло|\w+ет|\w+ется|\w+ешь|\w+ит|\w+ится|\w+лось|больше \w+ли|было|в порядке|вернуть|время|дело|для|еще \w+ли|еще \w+лись|еще|же|звонили колокола|знаю|зря|исправить|казалось|лето|неохотно|никак|нормально|понял|порти\w*|пространство|равно|растрепал\w*|ревел|сильнее|случайность|смотрел[ао]{1}|так же|такое|утро|хватали и хватали|чего|что|это)\b'
pattern2 = r'(\bдо\b =?)(\w+ей)'
re.findall(pattern, text)


[('громче и громче', 'громче', ' и громче')]

In [5]:
text = '- Третий закон Голпалотта гласит, - начала Гермиона, - осторожно, там может быть столик. Гласит, что противоядие от составного зелья - не скинь мою книгу, Мерлина ради - не сводится к смеси противоядий - не врежься в камин - от каждого его компонента.'
pattern = r', - начала(( \w+){0,3})\b'
re.findall(pattern, text)
re.sub(pattern, r', - началА \g<1>', text)
#for o in re.finditer(pattern, text):
#    print(o.start())
#    print(o.end())

'- Третий закон Голпалотта гласит, - началА  Гермиона, - осторожно, там может быть столик. Гласит, что противоядие от составного зелья - не скинь мою книгу, Мерлина ради - не сводится к смеси противоядий - не врежься в камин - от каждого его компонента.'

In [14]:
text1 = 'правильно и вообще'
text2 = 'правильно'
text3 = 'правильно. Пошли завтракать'

pattern = r'\bправильно\. \b'

re.findall(pattern, text3)

['правильно. ']